In [28]:
import json
import ast
import pandas as pd
from typing import Dict, List, Any
from pathlib import Path
import logging

def parse_json_safely(text: str) -> Dict:
    """
    Safely parse JSON string with multiple fallback methods
    """
    if not isinstance(text, str):
        return {}
    
    text = text.strip()
    if not text:
        return {}
        
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        try:
            return ast.literal_eval(text)
        except:
            return {}

def extract_car_details(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_detail section
    """
    result = {}
    
    # Direct fields we want to extract
    fields = ['ft', 'bt', 'km', 'transmission', 'ownerNo', 'owner', 
              'oem', 'model', 'modelYear','centralVariantId', 'variantName', 'price', 'priceFixedText', 'trendingText']
    
    for field in fields:
        if field in json_data:
            result[f'detail_{field}'] = json_data[field]
    
    return result

def extract_car_overview(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_overview section
    """
    result = {}
    
    if 'top' in json_data and isinstance(json_data['top'], list):
        for item in json_data['top']:
            if isinstance(item, dict) and 'key' in item and 'value' in item:
                key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                result[f'overview_{key}'] = item['value']
    
    return result

def extract_car_features(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_feature section
    """
    result = {}
    
    # Extract top features
    if 'top' in json_data and isinstance(json_data['top'], list):
        top_features = [item.get('value', '') for item in json_data['top'] if isinstance(item, dict)]
        result['feature_top'] = ', '.join(filter(None, top_features))
    
    # Extract detailed features by category
    if 'data' in json_data and isinstance(json_data['data'], list):
        for category in json_data['data']:
            if isinstance(category, dict) and 'heading' in category and 'list' in category:
                category_name = category['heading'].lower().replace(' & ', '_').replace(' ', '_')
                features = [item.get('value', '') for item in category['list'] if isinstance(item, dict)]
                result[f'feature_{category_name}'] = ', '.join(filter(None, features))
    
    return result

def extract_car_specs(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_specs section with proper handling of the nested structure
    """
    result = {}
    
    # Process top-level specifications if present
    if 'top' in json_data and isinstance(json_data['top'], list):
        for item in json_data['top']:
            if isinstance(item, dict) and 'key' in item and 'value' in item:
                clean_key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                result[f'specs_top_{clean_key}'] = item['value']
    
    # Process detailed specifications in the data section
    if 'data' in json_data and isinstance(json_data['data'], list):
        for category in json_data['data']:
            if isinstance(category, dict) and 'heading' in category and 'list' in category:
                # Clean the category name
                category_name = category['heading'].lower().replace(' & ', '_').replace(' ', '_')
                
                # Process the specifications list
                if isinstance(category['list'], list):
                    # Store each specification as an individual column
                    for item in category['list']:
                        if isinstance(item, dict) and 'key' in item and 'value' in item:
                            clean_key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                            result[f'specs_{category_name}_{clean_key}'] = item['value']
                    
                    # Also store the complete JSON for the category
                    result[f'specs_{category_name}_json'] = json.dumps(category['list'])
    
    return result

def process_excel_files(file_paths: List[str], output_csv: str) -> pd.DataFrame:
    """
    Process Excel files containing car data
    """
    all_data = []
    
    for file_path in file_paths:
        print(f"Processing {file_path}...")
        
        try:
            # Read Excel file
            df = pd.read_excel(file_path)
            
            # Process each row
            for idx, row in df.iterrows():
                processed_row = {
                    'file_name': Path(file_path).name,
                    'row_index': idx
                }
                
                try:
                    # Process specs data using the dedicated function
                    if 'new_car_specs' in row and pd.notna(row['new_car_specs']):
                        specs_data = row['new_car_specs']
                        if isinstance(specs_data, str):
                            try:
                                specs_dict = json.loads(specs_data)
                            except json.JSONDecodeError:
                                try:
                                    specs_dict = ast.literal_eval(specs_data)
                                except:
                                    print(f"Error parsing new_car_specs in row {idx}")
                                    specs_dict = {}
                        else:
                            specs_dict = specs_data
                            
                        if specs_dict:
                            specs_result = extract_car_specs(specs_dict)
                            processed_row.update(specs_result)
                    
                    # Process other JSON columns normally
                    if 'new_car_detail' in row:
                        json_data = parse_json_safely(row['new_car_detail'])
                        processed_row.update(extract_car_details(json_data))
                    
                    if 'new_car_overview' in row:
                        json_data = parse_json_safely(row['new_car_overview'])
                        processed_row.update(extract_car_overview(json_data))
                    
                    if 'new_car_feature' in row:
                        json_data = parse_json_safely(row['new_car_feature'])
                        processed_row.update(extract_car_features(json_data))
                    
                    # Add any non-JSON columns
                    for col in df.columns:
                        if col not in ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs']:
                            processed_row[col] = row[col]
                    
                except Exception as e:
                    print(f"Error processing row {idx}: {str(e)}")
                    continue
                
                all_data.append(processed_row)
                
        except Exception as e:
            print(f"Error processing file {file_path}: {str(e)}")
            continue
    
    # Create final DataFrame
    final_df = pd.DataFrame(all_data)
    
    # Save to CSV
    if output_csv:
        final_df.to_csv(output_csv, index=False)
        print(f"Saved processed data to {output_csv}")
        
        # Print column summary
        print("\nExtracted Columns:")
        categories = ['detail_', 'overview_', 'feature_', 'specs_']
        for prefix in categories:
            cols = [col for col in final_df.columns if col.startswith(prefix)]
            if cols:
                print(f"\n{prefix.upper()} columns:")
                for col in sorted(cols):
                    non_null_count = final_df[col].notna().sum()
                    print(f"- {col} ({non_null_count} non-null values)")
        
        print(f"\nTotal number of columns: {len(final_df.columns)}")
        print(f"Total number of rows: {len(final_df)}")
    
    return final_df

def main():
    """
    Main function to execute the processing
    """
    excel_files = [
        r"C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\bangalore_cars.xlsx"
    ]
    output_path = "processed_car_data_v5.csv"
    
    try:
        # Process files
        result_df = process_excel_files(excel_files, output_path)
        
        # Display sample of processed data
        print("\nSample of processed data:")
        print(result_df.head())
        
        # Display some basic statistics
        print("\nBasic Statistics:")
        print(f"Total cars processed: {len(result_df)}")
        if 'detail_price' in result_df.columns:
            print("\nPrice Distribution:")
            print(result_df['detail_price'].value_counts().head())
        
        if 'detail_modelYear' in result_df.columns:
            print("\nModel Year Distribution:")
            print(result_df['detail_modelYear'].value_counts().sort_index().head())
            
    except Exception as e:
        print(f"Error in main execution: {str(e)}")

if __name__ == "__main__":
    main()

Processing C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\bangalore_cars.xlsx...
Saved processed data to processed_car_data_v5.csv

Extracted Columns:

DETAIL_ columns:
- detail_bt (1481 non-null values)
- detail_centralVariantId (1481 non-null values)
- detail_ft (1481 non-null values)
- detail_km (1481 non-null values)
- detail_model (1481 non-null values)
- detail_modelYear (1481 non-null values)
- detail_oem (1481 non-null values)
- detail_owner (1481 non-null values)
- detail_ownerNo (1481 non-null values)
- detail_price (1481 non-null values)
- detail_priceFixedText (0 non-null values)
- detail_transmission (1481 non-null values)
- detail_trendingText (1481 non-null values)
- detail_variantName (1481 non-null values)

OVERVIEW_ columns:
- overview_engine_displacement (1478 non-null values)
- overview_fuel_type (1481 non-null values)
- overview_insurance_validity (1478 non-null values)
- overview_kms_driven (1481 non-null values)
- overview_ownership

In [2]:
import json
import ast
import pandas as pd
from typing import Dict, List, Any
from pathlib import Path
import logging

In [29]:

df_4= pd.read_csv("processed_car_data_v5.csv")
df_4.head()

,file_name,row_index,specs_top_mileage,specs_top_engine,specs_top_max_power,specs_top_torque,specs_top_seats,specs_engine_and_transmission_color,specs_engine_and_transmission_engine_type,specs_engine_and_transmission_displacement,...,feature_top,feature_comfort_convenience,feature_interior,feature_exterior,feature_safety,car_links,specs_top_wheel_size,specs_miscellaneous_alloy_wheel_size,feature_entertainment_communication,specs_dimensions_capacity_ground_clearance_unladen
0,bangalore_cars.xlsx,0,23.1 kmpl,998 CC,67.04bhp,90Nm,5.0,White,K10B Engine,998.0,...,"Power Steering, Power Windows Front, Air Condi...","Power Steering, Power Windows Front, Power Win...","Air Conditioner, Heater, Digital Odometer, Ele...","Adjustable Head Lights, Manually Adjustable Ex...","Centeral Locking, Child Safety Locks, Day Nigh...",https://www.cardekho.com/used-car-details/used...,NaN,NaN,NaN,NaN
1,bangalore_cars.xlsx,1,17 kmpl,1497 CC,121.31bhp,150Nm,5.0,White,Ti-VCT Petrol Engine,1497.0,...,"Power Steering, Power Windows Front, Air Condi...","Power Steering, Power Windows Front, Power Win...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe...","Anti Lock Braking System, Centeral Locking, Po...",https://www.cardekho.com/buy-used-car-details/...,16,16,"Cd Player, Radio, Speakers Front, Speakers Rea...",NaN
2,bangalore_cars.xlsx,2,23.84 kmpl,1199 CC,84bhp,114Nm,5.0,Red,Revotron Engine,1199.0,...,"Power Steering, Power Windows Front, Air Condi...","Power Steering, Power Windows Front, Power Win...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe...","Anti Lock Braking System, Centeral Locking, Po...",https://www.cardekho.com/used-car-details/used...,14,14,"Cd Player, Radio, Speakers Front, Speakers Rea...",NaN
3,bangalore_cars.xlsx,3,19.1 kmpl,1197 CC,81.86bhp,113.75Nm,5.0,Others,Kappa VTVT Petrol Engine,1197.0,...,"Power Steering, Power Windows Front, Air Condi...","Power Steering, Power Windows Front, Power Win...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe...","Anti Lock Braking System, Centeral Locking, Po...",https://www.cardekho.com/buy-used-car-details/...,14,14,"Cd Player, Radio, Speakers Front, Speakers Rea...",NaN
4,bangalore_cars.xlsx,4,23.65 kmpl,1248 CC,88.5bhp,200Nm,5.0,Gray,DDiS 200 Diesel Engine,1248.0,...,"Power Steering, Power Windows Front, Air Condi...","Power Steering, Power Windows Front, Power Win...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe...","Anti Lock Braking System, Centeral Locking, Po...",https://www.cardekho.com/used-car-details/used...,16,16,"Radio, Audio System Remote Control, Speakers F...",NaN


In [30]:
# Specify the full path with the filename, not just the directory
output_path = r"C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\processed_data.csv"

# Save the DataFrame to the specified path
df_4.to_csv(output_path, index=False)

print(f"CSV file has been exported to '{output_path}'")

CSV file has been exported to 'C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\processed_data.csv'


In [35]:

import json
import ast
import pandas as pd
from typing import Dict, List, Any
from pathlib import Path
import logging

def parse_json_safely(text: str) -> Dict:
    """
    Safely parse JSON string with multiple fallback methods
    """
    if not isinstance(text, str):
        return {}
    
    text = text.strip()
    if not text:
        return {}
        
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        try:
            return ast.literal_eval(text)
        except:
            return {}

def extract_car_details(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_detail section
    """
    result = {}
    
    # Direct fields we want to extract
    fields = ['ft', 'bt', 'km', 'transmission', 'ownerNo', 'owner', 
              'oem', 'model', 'modelYear','centralVariantId', 'variantName', 'price', 'priceFixedText', 'trendingText']
    
    for field in fields:
        if field in json_data:
            result[f'detail_{field}'] = json_data[field]
    
    return result

def extract_car_overview(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_overview section
    """
    result = {}
    
    if 'top' in json_data and isinstance(json_data['top'], list):
        for item in json_data['top']:
            if isinstance(item, dict) and 'key' in item and 'value' in item:
                key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                result[f'overview_{key}'] = item['value']
    
    return result

def extract_car_features(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_feature section
    """
    result = {}
    
    # Extract top features
    if 'top' in json_data and isinstance(json_data['top'], list):
        top_features = [item.get('value', '') for item in json_data['top'] if isinstance(item, dict)]
        result['feature_top'] = ', '.join(filter(None, top_features))
    
    # Extract detailed features by category
    if 'data' in json_data and isinstance(json_data['data'], list):
        for category in json_data['data']:
            if isinstance(category, dict) and 'heading' in category and 'list' in category:
                category_name = category['heading'].lower().replace(' & ', '_').replace(' ', '_')
                features = [item.get('value', '') for item in category['list'] if isinstance(item, dict)]
                result[f'feature_{category_name}'] = ', '.join(filter(None, features))
    
    return result

def extract_car_specs(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_specs section with proper handling of the nested structure
    """
    result = {}
    
    # Process top-level specifications if present
    if 'top' in json_data and isinstance(json_data['top'], list):
        for item in json_data['top']:
            if isinstance(item, dict) and 'key' in item and 'value' in item:
                clean_key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                result[f'specs_top_{clean_key}'] = item['value']
    
    # Process detailed specifications in the data section
    if 'data' in json_data and isinstance(json_data['data'], list):
        for category in json_data['data']:
            if isinstance(category, dict) and 'heading' in category and 'list' in category:
                # Clean the category name
                category_name = category['heading'].lower().replace(' & ', '_').replace(' ', '_')
                
                # Process the specifications list
                if isinstance(category['list'], list):
                    # Store each specification as an individual column
                    for item in category['list']:
                        if isinstance(item, dict) and 'key' in item and 'value' in item:
                            clean_key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                            result[f'specs_{category_name}_{clean_key}'] = item['value']
                    
                    # Also store the complete JSON for the category
                    result[f'specs_{category_name}_json'] = json.dumps(category['list'])
    
    return result

def process_excel_files(file_paths: List[str], output_csv: str) -> pd.DataFrame:
    """
    Process Excel files containing car data with location verification
    """
    all_data = []
    
    for file_path in file_paths:
        print(f"\nProcessing {file_path}...")
        
        try:
            # Read Excel file
            df = pd.read_excel(file_path)
            
            # Print initial data info
            print(f"Initial data shape: {df.shape}")
            
            # Check if 'city' column exists and print unique cities
            if 'city' in df.columns:
                print("Cities in dataset:", df['city'].unique())
            elif 'location' in df.columns:
                print("Locations in dataset:", df['location'].unique())
            
            # Process each row
            for idx, row in df.iterrows():
                processed_row = {
                    'file_name': Path(file_path).name,
                    'row_index': idx
                }
                
                try:
                    # Store location information if available
                    if 'city' in row:
                        processed_row['city'] = row['city']
                    if 'location' in row:
                        processed_row['location'] = row['location']
                    
                    # Process specs data using the dedicated function
                    if 'new_car_specs' in row and pd.notna(row['new_car_specs']):
                        specs_data = row['new_car_specs']
                        if isinstance(specs_data, str):
                            try:
                                specs_dict = json.loads(specs_data)
                            except json.JSONDecodeError:
                                try:
                                    specs_dict = ast.literal_eval(specs_data)
                                except:
                                    print(f"Error parsing new_car_specs in row {idx}")
                                    specs_dict = {}
                        else:
                            specs_dict = specs_data
                            
                        if specs_dict:
                            specs_result = extract_car_specs(specs_dict)
                            processed_row.update(specs_result)
                    
                    # Process car details and extract location if present
                    if 'new_car_detail' in row:
                        json_data = parse_json_safely(row['new_car_detail'])
                        detail_result = extract_car_details(json_data)
                        processed_row.update(detail_result)
                        
                        # Check if location info is in car details
                        if 'city' in json_data:
                            processed_row['detail_city'] = json_data['city']
                        if 'location' in json_data:
                            processed_row['detail_location'] = json_data['location']
                    
                    if 'new_car_overview' in row:
                        json_data = parse_json_safely(row['new_car_overview'])
                        processed_row.update(extract_car_overview(json_data))
                    
                    if 'new_car_feature' in row:
                        json_data = parse_json_safely(row['new_car_feature'])
                        processed_row.update(extract_car_features(json_data))
                    
                    # Add any non-JSON columns
                    for col in df.columns:
                        if col not in ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs']:
                            processed_row[col] = row[col]
                    
                except Exception as e:
                    print(f"Error processing row {idx}: {str(e)}")
                    continue
                
                all_data.append(processed_row)
                
        except Exception as e:
            print(f"Error processing file {file_path}: {str(e)}")
            continue
    
    # Create final DataFrame
    final_df = pd.DataFrame(all_data)
    
    # Verify location data
    print("\nLocation verification:")
    location_columns = ['city', 'location', 'detail_city', 'detail_location']
    for col in location_columns:
        if col in final_df.columns:
            print(f"\nUnique values in {col}:")
            print(final_df[col].value_counts().head())
    
    # Save to CSV
    if output_csv:
        final_df.to_csv(output_csv, index=False)
        print(f"\nSaved processed data to {output_csv}")
        
        # Print column summary
        print("\nExtracted Columns:")
        categories = ['detail_', 'overview_', 'feature_', 'specs_']
        for prefix in categories:
            cols = [col for col in final_df.columns if col.startswith(prefix)]
            if cols:
                print(f"\n{prefix.upper()} columns:")
                for col in sorted(cols):
                    non_null_count = final_df[col].notna().sum()
                    print(f"- {col} ({non_null_count} non-null values)")
        
        print(f"\nTotal number of columns: {len(final_df.columns)}")
        print(f"Total number of rows: {len(final_df)}")
    
    return final_df

def main():
    """
    Main function to execute the processing with additional verification
    """
    excel_files = [
        r"C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\chennai_cars.xlsx"
    ]
    output_path = "processed_car_data_chennai.csv"
    
    try:
        # Process files
        result_df = process_excel_files(excel_files, output_path)
        
        # Verify location data
        print("\nFinal Location Verification:")
        location_cols = ['city', 'location', 'detail_city', 'detail_location']
        for col in location_cols:
            if col in result_df.columns:
                print(f"\nDistribution of {col}:")
                print(result_df[col].value_counts().head())
        
        # Display sample of processed data
        print("\nSample of processed data:")
        print(result_df.head())
        
        # Display some basic statistics
        print("\nBasic Statistics:")
        print(f"Total cars processed: {len(result_df)}")
        if 'detail_price' in result_df.columns:
            print("\nPrice Distribution:")
            print(result_df['detail_price'].value_counts().head())
        
        if 'detail_modelYear' in result_df.columns:
            print("\nModel Year Distribution:")
            print(result_df['detail_modelYear'].value_counts().sort_index().head())
            
    except Exception as e:
        print(f"Error in main execution: {str(e)}")

if __name__ == "__main__":
    main()





Processing C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\chennai_cars.xlsx...
Initial data shape: (1419, 5)

Location verification:

Saved processed data to processed_car_data_chennai.csv

Extracted Columns:

DETAIL_ columns:
- detail_bt (1419 non-null values)
- detail_centralVariantId (1419 non-null values)
- detail_ft (1419 non-null values)
- detail_km (1419 non-null values)
- detail_model (1419 non-null values)
- detail_modelYear (1419 non-null values)
- detail_oem (1419 non-null values)
- detail_owner (1419 non-null values)
- detail_ownerNo (1419 non-null values)
- detail_price (1419 non-null values)
- detail_priceFixedText (0 non-null values)
- detail_transmission (1419 non-null values)
- detail_trendingText (1419 non-null values)
- detail_variantName (1419 non-null values)

OVERVIEW_ columns:
- overview_engine_displacement (1419 non-null values)
- overview_fuel_type (1419 non-null values)
- overview_insurance_validity (1419 non-null values)
- over

In [36]:
df_chennai= pd.read_csv("processed_car_data_chennai.csv")
df_chennai.head()

,file_name,row_index,specs_top_engine,specs_top_max_power,specs_top_torque,specs_top_wheel_size,specs_top_seats,specs_engine_and_transmission_color,specs_engine_and_transmission_engine_type,specs_engine_and_transmission_displacement,...,specs_engine_and_transmission_compression_ratio,specs_engine_and_transmission_super_charger,specs_dimensions_capacity_front_tread,specs_dimensions_capacity_rear_tread,specs_dimensions_capacity_gross_weight,specs_miscellaneous_turning_radius,specs_miscellaneous_top_speed,specs_miscellaneous_acceleration,specs_engine_and_transmission_borex_stroke,specs_dimensions_capacity_ground_clearance_unladen
0,chennai_cars.xlsx,0,998 CC,118.36bhp,172nm,16,5,Black,Smartstream G1.0 T - GDi,998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chennai_cars.xlsx,1,1196 CC,73bhp,101Nm,NaN,7,Grey,In-Line Engine,1196,...,9.9:1,No,1280mm,1290mm,1540kg,4.5 metres,145 Kmph,15.7 Seconds,NaN,NaN
2,chennai_cars.xlsx,2,999 CC,98.63bhp,160Nm,16,5,Others,HRA0 1.0 TURBO PETROL,999,...,NaN,No,NaN,NaN,NaN,5.0,NaN,11.7,72.2 x 81.3,NaN
3,chennai_cars.xlsx,3,1086 CC,68.05bhp,99.04Nm,NaN,5,Silver,IRDE2 Engine,1086,...,NaN,No,1400mm,1385mm,NaN,4.7 metres,165 Kmph,14.3 Seconds,NaN,NaN
4,chennai_cars.xlsx,4,1199 CC,88.7bhp,110Nm,15,5,Others,i-VTEC Petrol Engine,1199,...,NaN,No,NaN,NaN,NaN,5.1 meters,172 Kmph,13.7 Seconds,NaN,NaN


In [37]:
# Specify the full path with the filename, not just the directory
output_path = r"C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\processed_car_data_chennai.csv"

# Save the DataFrame to the specified path
df_chennai.to_csv(output_path, index=False)

print(f"CSV file has been exported to '{output_path}'")

CSV file has been exported to 'C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\processed_car_data_chennai.csv'


In [3]:
def parse_json_safely(text: str) -> Dict:
    """
    Safely parse JSON string with multiple fallback methods
    """
    if not isinstance(text, str):
        return {}
    
    text = text.strip()
    if not text:
        return {}
        
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        try:
            return ast.literal_eval(text)
        except:
            return {}

def extract_car_details(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_detail section
    """
    result = {}
    
    # Direct fields we want to extract
    fields = ['ft', 'bt', 'km', 'transmission', 'ownerNo', 'owner', 
              'oem', 'model', 'modelYear','centralVariantId', 'variantName', 'price', 'priceFixedText', 'trendingText']
    
    for field in fields:
        if field in json_data:
            result[f'detail_{field}'] = json_data[field]
    
    return result

def extract_car_overview(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_overview section
    """
    result = {}
    
    if 'top' in json_data and isinstance(json_data['top'], list):
        for item in json_data['top']:
            if isinstance(item, dict) and 'key' in item and 'value' in item:
                key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                result[f'overview_{key}'] = item['value']
    
    return result

def extract_car_features(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_feature section
    """
    result = {}
    
    # Extract top features
    if 'top' in json_data and isinstance(json_data['top'], list):
        top_features = [item.get('value', '') for item in json_data['top'] if isinstance(item, dict)]
        result['feature_top'] = ', '.join(filter(None, top_features))
    
    # Extract detailed features by category
    if 'data' in json_data and isinstance(json_data['data'], list):
        for category in json_data['data']:
            if isinstance(category, dict) and 'heading' in category and 'list' in category:
                category_name = category['heading'].lower().replace(' & ', '_').replace(' ', '_')
                features = [item.get('value', '') for item in category['list'] if isinstance(item, dict)]
                result[f'feature_{category_name}'] = ', '.join(filter(None, features))
    
    return result

def extract_car_specs(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_specs section with proper handling of the nested structure
    """
    result = {}
    
    # Process top-level specifications if present
    if 'top' in json_data and isinstance(json_data['top'], list):
        for item in json_data['top']:
            if isinstance(item, dict) and 'key' in item and 'value' in item:
                clean_key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                result[f'specs_top_{clean_key}'] = item['value']
    
    # Process detailed specifications in the data section
    if 'data' in json_data and isinstance(json_data['data'], list):
        for category in json_data['data']:
            if isinstance(category, dict) and 'heading' in category and 'list' in category:
                # Clean the category name
                category_name = category['heading'].lower().replace(' & ', '_').replace(' ', '_')
                
                # Process the specifications list
                if isinstance(category['list'], list):
                    # Store each specification as an individual column
                    for item in category['list']:
                        if isinstance(item, dict) and 'key' in item and 'value' in item:
                            clean_key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                            result[f'specs_{category_name}_{clean_key}'] = item['value']
                    
                    # Also store the complete JSON for the category
                    result[f'specs_{category_name}_json'] = json.dumps(category['list'])
    
    return result

def process_excel_files(file_paths: List[str], output_csv: str) -> pd.DataFrame:
    """
    Process Excel files containing car data with location verification
    """
    all_data = []
    
    for file_path in file_paths:
        # Extract city name from file path
        city_name = Path(file_path).stem.split('_')[0].capitalize()
        print(f"\nProcessing {city_name} data from {file_path}...")
        
        try:
            # Read Excel file
            df = pd.read_excel(file_path)
            print(f"Initial data shape for {city_name}: {df.shape}")
            
            # Add city column if not present
            if 'city' not in df.columns:
                df['city'] = city_name
            
            # Process each row
            for idx, row in df.iterrows():
                processed_row = {
                    'file_name': Path(file_path).name,
                    'row_index': idx,
                    'source_city': city_name  # Add explicit source city tracking
                }
                
                try:
                    # Store location information
                    processed_row['city'] = row.get('city', city_name)
                    if 'location' in row:
                        processed_row['location'] = row['location']
                    
                    # Process specs data
                    if 'new_car_specs' in row and pd.notna(row['new_car_specs']):
                        specs_data = row['new_car_specs']
                        if isinstance(specs_data, str):
                            try:
                                specs_dict = json.loads(specs_data)
                            except json.JSONDecodeError:
                                try:
                                    specs_dict = ast.literal_eval(specs_data)
                                except:
                                    print(f"Error parsing new_car_specs in row {idx}")
                                    specs_dict = {}
                        else:
                            specs_dict = specs_data
                            
                        if specs_dict:
                            specs_result = extract_car_specs(specs_dict)
                            processed_row.update(specs_result)
                    
                    # Process other JSON columns
                    if 'new_car_detail' in row:
                        json_data = parse_json_safely(row['new_car_detail'])
                        processed_row.update(extract_car_details(json_data))
                    
                    if 'new_car_overview' in row:
                        json_data = parse_json_safely(row['new_car_overview'])
                        processed_row.update(extract_car_overview(json_data))
                    
                    if 'new_car_feature' in row:
                        json_data = parse_json_safely(row['new_car_feature'])
                        processed_row.update(extract_car_features(json_data))
                    
                    # Add any non-JSON columns
                    for col in df.columns:
                        if col not in ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs']:
                            processed_row[col] = row[col]
                    
                except Exception as e:
                    print(f"Error processing row {idx} in {city_name} data: {str(e)}")
                    continue
                
                all_data.append(processed_row)
                
        except Exception as e:
            print(f"Error processing file {file_path}: {str(e)}")
            continue
    
    # Create final DataFrame
    final_df = pd.DataFrame(all_data)
    
    # Verify data by city
    print("\nData verification by city:")
    print(final_df['source_city'].value_counts())
    
    # Save to CSV
    if output_csv:
        final_df.to_csv(output_csv, index=False)
        print(f"\nSaved processed data to {output_csv}")
        
        # Print column summary by city
        print("\nColumn Summary by City:")
        for city in final_df['source_city'].unique():
            city_df = final_df[final_df['source_city'] == city]
            print(f"\n{city} Data Summary:")
            print(f"Total rows: {len(city_df)}")
            
            categories = ['detail_', 'overview_', 'feature_', 'specs_']
            for prefix in categories:
                cols = [col for col in city_df.columns if col.startswith(prefix)]
                if cols:
                    print(f"\n{prefix.upper()} columns in {city}:")
                    for col in sorted(cols):
                        non_null_count = city_df[col].notna().sum()
                        print(f"- {col} ({non_null_count} non-null values)")
    
    return final_df

def main():
    """
    Main function to execute the processing for multiple cities
    """
    excel_files = [
        r"C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\delhi_cars.xlsx"
    ]
    output_path = "processed_car_data_delhi.csv"
    
    try:
        # Process files
        result_df = process_excel_files(excel_files, output_path)
        
        # Display statistics by city
        for city in result_df['source_city'].unique():
            city_df = result_df[result_df['source_city'] == city]
            print(f"\nStatistics for {city}:")
            print(f"Total cars: {len(city_df)}")
            
            if 'detail_price' in city_df.columns:
                print(f"\nPrice Distribution in {city}:")
                print(city_df['detail_price'].value_counts().head())
            
            if 'detail_modelYear' in city_df.columns:
                print(f"\nModel Year Distribution in {city}:")
                print(city_df['detail_modelYear'].value_counts().sort_index().head())
            
            if 'detail_transmission' in city_df.columns:
                print(f"\nTransmission Types in {city}:")
                print(city_df['detail_transmission'].value_counts())
            
            print("\n" + "="*50)
        
        # Save separate CSV files for each city
        for city in result_df['source_city'].unique():
            city_df = result_df[result_df['source_city'] == city]
            city_output = f"processed_car_data_{city.lower()}.csv"
            city_df.to_csv(city_output, index=False)
            print(f"\nSaved {city} data to {city_output}")
            
    except Exception as e:
        print(f"Error in main execution: {str(e)}")

if __name__ == "__main__":
    main()


Processing Delhi data from C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\delhi_cars.xlsx...
Initial data shape for Delhi: (1485, 5)

Data verification by city:
source_city
Delhi    1485
Name: count, dtype: int64

Saved processed data to processed_car_data_delhi.csv

Column Summary by City:

Delhi Data Summary:
Total rows: 1485

DETAIL_ columns in Delhi:
- detail_bt (1485 non-null values)
- detail_centralVariantId (1485 non-null values)
- detail_ft (1485 non-null values)
- detail_km (1485 non-null values)
- detail_model (1485 non-null values)
- detail_modelYear (1485 non-null values)
- detail_oem (1485 non-null values)
- detail_owner (1485 non-null values)
- detail_ownerNo (1485 non-null values)
- detail_price (1485 non-null values)
- detail_priceFixedText (0 non-null values)
- detail_transmission (1485 non-null values)
- detail_trendingText (1485 non-null values)
- detail_variantName (1485 non-null values)

OVERVIEW_ columns in Delhi:
- overview_engine_

In [4]:
df_delhi= pd.read_csv("processed_car_data_delhi.csv")
df_delhi.head()

,file_name,row_index,source_city,city,specs_top_mileage,specs_top_engine,specs_top_max_power,specs_top_torque,specs_top_wheel_size,specs_top_seats,...,specs_miscellaneous_turning_radius,specs_miscellaneous_top_speed,specs_miscellaneous_acceleration,overview_rto,specs_dimensions_capacity_gross_weight,specs_dimensions_capacity_front_tread,specs_dimensions_capacity_rear_tread,specs_engine_and_transmission_borex_stroke,specs_engine_and_transmission_compression_ratio,specs_dimensions_capacity_ground_clearance_unladen
0,delhi_cars.xlsx,0,Delhi,Delhi,18 kmpl,1493 CC,113.43bhp,250Nm,17,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,delhi_cars.xlsx,1,Delhi,Delhi,15.29 kmpl,1591 CC,121.3bhp,151Nm,NaN,5.0,...,5.3 metres,165 Kmph,10.5 Seconds,HR51,NaN,NaN,NaN,NaN,NaN,NaN
2,delhi_cars.xlsx,2,Delhi,Delhi,12.74 kmpl,1991 CC,197bhp,320nm,19,5.0,...,NaN,217 Kmph,NaN,DL01,2360kg,NaN,NaN,NaN,NaN,NaN
3,delhi_cars.xlsx,3,Delhi,Delhi,22.38 kmpl,1197 CC,88.50bhp,113Nm,NaN,5.0,...,4.8,NaN,NaN,DL4C,1335,1530,1530,NaN,NaN,NaN
4,delhi_cars.xlsx,4,Delhi,Delhi,15.29 kmpl,1591 CC,121.3bhp,151Nm,NaN,5.0,...,5.3 metres,165 Kmph,10.5 Seconds,DL8C,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Specify the full path with the filename, not just the directory
output_path = r"C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\processed_car_data_delhi.csv"

# Save the DataFrame to the specified path
df_delhi.to_csv(output_path, index=False)

print(f"CSV file has been exported to '{output_path}'")

CSV file has been exported to 'C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\processed_car_data_delhi.csv'


In [6]:
def parse_json_safely(text: str) -> Dict:
    """
    Safely parse JSON string with multiple fallback methods
    """
    if not isinstance(text, str):
        return {}
    
    text = text.strip()
    if not text:
        return {}
        
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        try:
            return ast.literal_eval(text)
        except:
            return {}

def extract_car_details(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_detail section
    """
    result = {}
    
    # Direct fields we want to extract
    fields = ['ft', 'bt', 'km', 'transmission', 'ownerNo', 'owner', 
              'oem', 'model', 'modelYear','centralVariantId', 'variantName', 'price', 'priceFixedText', 'trendingText']
    
    for field in fields:
        if field in json_data:
            result[f'detail_{field}'] = json_data[field]
    
    return result

def extract_car_overview(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_overview section
    """
    result = {}
    
    if 'top' in json_data and isinstance(json_data['top'], list):
        for item in json_data['top']:
            if isinstance(item, dict) and 'key' in item and 'value' in item:
                key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                result[f'overview_{key}'] = item['value']
    
    return result

def extract_car_features(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_feature section
    """
    result = {}
    
    # Extract top features
    if 'top' in json_data and isinstance(json_data['top'], list):
        top_features = [item.get('value', '') for item in json_data['top'] if isinstance(item, dict)]
        result['feature_top'] = ', '.join(filter(None, top_features))
    
    # Extract detailed features by category
    if 'data' in json_data and isinstance(json_data['data'], list):
        for category in json_data['data']:
            if isinstance(category, dict) and 'heading' in category and 'list' in category:
                category_name = category['heading'].lower().replace(' & ', '_').replace(' ', '_')
                features = [item.get('value', '') for item in category['list'] if isinstance(item, dict)]
                result[f'feature_{category_name}'] = ', '.join(filter(None, features))
    
    return result

def extract_car_specs(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_specs section with proper handling of the nested structure
    """
    result = {}
    
    # Process top-level specifications if present
    if 'top' in json_data and isinstance(json_data['top'], list):
        for item in json_data['top']:
            if isinstance(item, dict) and 'key' in item and 'value' in item:
                clean_key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                result[f'specs_top_{clean_key}'] = item['value']
    
    # Process detailed specifications in the data section
    if 'data' in json_data and isinstance(json_data['data'], list):
        for category in json_data['data']:
            if isinstance(category, dict) and 'heading' in category and 'list' in category:
                # Clean the category name
                category_name = category['heading'].lower().replace(' & ', '_').replace(' ', '_')
                
                # Process the specifications list
                if isinstance(category['list'], list):
                    # Store each specification as an individual column
                    for item in category['list']:
                        if isinstance(item, dict) and 'key' in item and 'value' in item:
                            clean_key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                            result[f'specs_{category_name}_{clean_key}'] = item['value']
                    
                    # Also store the complete JSON for the category
                    result[f'specs_{category_name}_json'] = json.dumps(category['list'])
    
    return result

def process_excel_files(file_paths: List[str], output_csv: str) -> pd.DataFrame:
    """
    Process Excel files containing car data with location verification
    """
    all_data = []
    
    for file_path in file_paths:
        # Extract city name from file path
        city_name = Path(file_path).stem.split('_')[0].capitalize()
        print(f"\nProcessing {city_name} data from {file_path}...")
        
        try:
            # Read Excel file
            df = pd.read_excel(file_path)
            print(f"Initial data shape for {city_name}: {df.shape}")
            
            # Add city column if not present
            if 'city' not in df.columns:
                df['city'] = city_name
            
            # Process each row
            for idx, row in df.iterrows():
                processed_row = {
                    'file_name': Path(file_path).name,
                    'row_index': idx,
                    'source_city': city_name  # Add explicit source city tracking
                }
                
                try:
                    # Store location information
                    processed_row['city'] = row.get('city', city_name)
                    if 'location' in row:
                        processed_row['location'] = row['location']
                    
                    # Process specs data
                    if 'new_car_specs' in row and pd.notna(row['new_car_specs']):
                        specs_data = row['new_car_specs']
                        if isinstance(specs_data, str):
                            try:
                                specs_dict = json.loads(specs_data)
                            except json.JSONDecodeError:
                                try:
                                    specs_dict = ast.literal_eval(specs_data)
                                except:
                                    print(f"Error parsing new_car_specs in row {idx}")
                                    specs_dict = {}
                        else:
                            specs_dict = specs_data
                            
                        if specs_dict:
                            specs_result = extract_car_specs(specs_dict)
                            processed_row.update(specs_result)
                    
                    # Process other JSON columns
                    if 'new_car_detail' in row:
                        json_data = parse_json_safely(row['new_car_detail'])
                        processed_row.update(extract_car_details(json_data))
                    
                    if 'new_car_overview' in row:
                        json_data = parse_json_safely(row['new_car_overview'])
                        processed_row.update(extract_car_overview(json_data))
                    
                    if 'new_car_feature' in row:
                        json_data = parse_json_safely(row['new_car_feature'])
                        processed_row.update(extract_car_features(json_data))
                    
                    # Add any non-JSON columns
                    for col in df.columns:
                        if col not in ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs']:
                            processed_row[col] = row[col]
                    
                except Exception as e:
                    print(f"Error processing row {idx} in {city_name} data: {str(e)}")
                    continue
                
                all_data.append(processed_row)
                
        except Exception as e:
            print(f"Error processing file {file_path}: {str(e)}")
            continue
    
    # Create final DataFrame
    final_df = pd.DataFrame(all_data)
    
    # Verify data by city
    print("\nData verification by city:")
    print(final_df['source_city'].value_counts())
    
    # Save to CSV
    if output_csv:
        final_df.to_csv(output_csv, index=False)
        print(f"\nSaved processed data to {output_csv}")
        
        # Print column summary by city
        print("\nColumn Summary by City:")
        for city in final_df['source_city'].unique():
            city_df = final_df[final_df['source_city'] == city]
            print(f"\n{city} Data Summary:")
            print(f"Total rows: {len(city_df)}")
            
            categories = ['detail_', 'overview_', 'feature_', 'specs_']
            for prefix in categories:
                cols = [col for col in city_df.columns if col.startswith(prefix)]
                if cols:
                    print(f"\n{prefix.upper()} columns in {city}:")
                    for col in sorted(cols):
                        non_null_count = city_df[col].notna().sum()
                        print(f"- {col} ({non_null_count} non-null values)")
    
    return final_df

def main():
    """
    Main function to execute the processing for multiple cities
    """
    excel_files = [
        r"C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\hyderabad_cars.xlsx" 
    ]
    output_path = "processed_car_data_hyderabad.csv"
    
    try:
        # Process files
        result_df = process_excel_files(excel_files, output_path)
        
        # Display statistics by city
        for city in result_df['source_city'].unique():
            city_df = result_df[result_df['source_city'] == city]
            print(f"\nStatistics for {city}:")
            print(f"Total cars: {len(city_df)}")
            
            if 'detail_price' in city_df.columns:
                print(f"\nPrice Distribution in {city}:")
                print(city_df['detail_price'].value_counts().head())
            
            if 'detail_modelYear' in city_df.columns:
                print(f"\nModel Year Distribution in {city}:")
                print(city_df['detail_modelYear'].value_counts().sort_index().head())
            
            if 'detail_transmission' in city_df.columns:
                print(f"\nTransmission Types in {city}:")
                print(city_df['detail_transmission'].value_counts())
            
            print("\n" + "="*50)
        
        # Save separate CSV files for each city
        for city in result_df['source_city'].unique():
            city_df = result_df[result_df['source_city'] == city]
            city_output = f"processed_car_data_{city.lower()}.csv"
            city_df.to_csv(city_output, index=False)
            print(f"\nSaved {city} data to {city_output}")
            
    except Exception as e:
        print(f"Error in main execution: {str(e)}")

if __name__ == "__main__":
    main()


Processing Hyderabad data from C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\hyderabad_cars.xlsx...
Initial data shape for Hyderabad: (1483, 5)

Data verification by city:
source_city
Hyderabad    1483
Name: count, dtype: int64

Saved processed data to processed_car_data_hyderabad.csv

Column Summary by City:

Hyderabad Data Summary:
Total rows: 1483

DETAIL_ columns in Hyderabad:
- detail_bt (1483 non-null values)
- detail_centralVariantId (1483 non-null values)
- detail_ft (1483 non-null values)
- detail_km (1483 non-null values)
- detail_model (1483 non-null values)
- detail_modelYear (1483 non-null values)
- detail_oem (1483 non-null values)
- detail_owner (1483 non-null values)
- detail_ownerNo (1483 non-null values)
- detail_price (1483 non-null values)
- detail_priceFixedText (0 non-null values)
- detail_transmission (1483 non-null values)
- detail_trendingText (1483 non-null values)
- detail_variantName (1483 non-null values)

OVERVIEW_ columns 

In [7]:
df_hyderabad= pd.read_csv("processed_car_data_hyderabad.csv")
df_hyderabad.head()

,file_name,row_index,source_city,city,specs_top_mileage,specs_top_engine,specs_top_max_power,specs_top_torque,specs_top_wheel_size,specs_top_seats,...,feature_top,feature_comfort_convenience,feature_interior,feature_exterior,feature_safety,feature_entertainment_communication,car_links,specs_miscellaneous_top_speed,specs_miscellaneous_acceleration,specs_dimensions_capacity_ground_clearance_unladen
0,hyderabad_cars.xlsx,0,Hyderabad,Hyderabad,16.47 kmpl,999 CC,108.62bhp,175nm,R16,5,...,"Power Steering, Power Windows Front, Air Condi...","Power Steering, Power Windows Front, Power Win...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Fog ...","Anti Lock Braking System, Centeral Locking, Po...","Radio, Speakers Front, Speakers Rear, Usb Auxi...",https://www.cardekho.com/used-car-details/used...,NaN,NaN,NaN
1,hyderabad_cars.xlsx,1,Hyderabad,Hyderabad,21.1 kmpl,814 CC,55.2bhp,74.5Nm,NaN,5,...,"Power Steering, Power Windows Front, Air Condi...","Power Steering, Power Windows Front, Remote Tr...","Air Conditioner, Heater, Digital Odometer, Ele...","Adjustable Head Lights, Manually Adjustable Ex...","Child Safety Locks, Driver Air Bag, Day Night ...",NaN,https://www.cardekho.com/buy-used-car-details/...,135 Kmph,19 Seconds,NaN
2,hyderabad_cars.xlsx,2,Hyderabad,Hyderabad,NaN,1197 CC,81.80bhp,113.8Nm,NaN,5,...,"Power Steering, Power Windows Front, Air Condi...","Power Steering, Power Windows Front, Power Win...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Power Adjustable Exter...","Anti Lock Braking System, Brake Assist, Center...","Radio, Speakers Front, Speakers Rear, Integrat...",https://www.cardekho.com/used-car-details/used...,NaN,NaN,NaN
3,hyderabad_cars.xlsx,3,Hyderabad,Hyderabad,21.4 kmpl,1197 CC,83.1bhp,115Nm,16,5,...,"Power Steering, Power Windows Front, Air Condi...","Power Steering, Power Windows Front, Power Win...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe...","Anti Lock Braking System, Centeral Locking, Po...","Radio, Audio System Remote Control, Speakers F...",https://www.cardekho.com/buy-used-car-details/...,180 Kmph,12.36 seconds,NaN
4,hyderabad_cars.xlsx,4,Hyderabad,Hyderabad,NaN,1997 CC,150bhp,320Nm,18,4,...,"Power Steering, Power Windows Front, Air Condi...","Power Steering, Power Windows Front, Low Fuel ...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe...","Anti Lock Braking System, Brake Assist, Center...","Radio, Speakers Front, Speakers Rear, Integrat...",https://www.cardekho.com/used-car-details/used...,NaN,NaN,NaN


In [8]:
# Specify the full path with the filename, not just the directory
output_path = r"C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\processed_car_data_hyderabad.csv"

# Save the DataFrame to the specified path
df_hyderabad.to_csv(output_path, index=False)

print(f"CSV file has been exported to '{output_path}'")

CSV file has been exported to 'C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\processed_car_data_hyderabad.csv'


In [10]:
def parse_json_safely(text: str) -> Dict:
    """
    Safely parse JSON string with multiple fallback methods
    """
    if not isinstance(text, str):
        return {}
    
    text = text.strip()
    if not text:
        return {}
        
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        try:
            return ast.literal_eval(text)
        except:
            return {}

def extract_car_details(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_detail section
    """
    result = {}
    
    # Direct fields we want to extract
    fields = ['ft', 'bt', 'km', 'transmission', 'ownerNo', 'owner', 
              'oem', 'model', 'modelYear','centralVariantId', 'variantName', 'price', 'priceFixedText', 'trendingText']
    
    for field in fields:
        if field in json_data:
            result[f'detail_{field}'] = json_data[field]
    
    return result

def extract_car_overview(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_overview section
    """
    result = {}
    
    if 'top' in json_data and isinstance(json_data['top'], list):
        for item in json_data['top']:
            if isinstance(item, dict) and 'key' in item and 'value' in item:
                key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                result[f'overview_{key}'] = item['value']
    
    return result

def extract_car_features(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_feature section
    """
    result = {}
    
    # Extract top features
    if 'top' in json_data and isinstance(json_data['top'], list):
        top_features = [item.get('value', '') for item in json_data['top'] if isinstance(item, dict)]
        result['feature_top'] = ', '.join(filter(None, top_features))
    
    # Extract detailed features by category
    if 'data' in json_data and isinstance(json_data['data'], list):
        for category in json_data['data']:
            if isinstance(category, dict) and 'heading' in category and 'list' in category:
                category_name = category['heading'].lower().replace(' & ', '_').replace(' ', '_')
                features = [item.get('value', '') for item in category['list'] if isinstance(item, dict)]
                result[f'feature_{category_name}'] = ', '.join(filter(None, features))
    
    return result

def extract_car_specs(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_specs section with proper handling of the nested structure
    """
    result = {}
    
    # Process top-level specifications if present
    if 'top' in json_data and isinstance(json_data['top'], list):
        for item in json_data['top']:
            if isinstance(item, dict) and 'key' in item and 'value' in item:
                clean_key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                result[f'specs_top_{clean_key}'] = item['value']
    
    # Process detailed specifications in the data section
    if 'data' in json_data and isinstance(json_data['data'], list):
        for category in json_data['data']:
            if isinstance(category, dict) and 'heading' in category and 'list' in category:
                # Clean the category name
                category_name = category['heading'].lower().replace(' & ', '_').replace(' ', '_')
                
                # Process the specifications list
                if isinstance(category['list'], list):
                    # Store each specification as an individual column
                    for item in category['list']:
                        if isinstance(item, dict) and 'key' in item and 'value' in item:
                            clean_key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                            result[f'specs_{category_name}_{clean_key}'] = item['value']
                    
                    # Also store the complete JSON for the category
                    result[f'specs_{category_name}_json'] = json.dumps(category['list'])
    
    return result

def process_excel_files(file_paths: List[str], output_csv: str) -> pd.DataFrame:
    """
    Process Excel files containing car data with location verification
    """
    all_data = []
    
    for file_path in file_paths:
        # Extract city name from file path
        city_name = Path(file_path).stem.split('_')[0].capitalize()
        print(f"\nProcessing {city_name} data from {file_path}...")
        
        try:
            # Read Excel file
            df = pd.read_excel(file_path)
            print(f"Initial data shape for {city_name}: {df.shape}")
            
            # Add city column if not present
            if 'city' not in df.columns:
                df['city'] = city_name
            
            # Process each row
            for idx, row in df.iterrows():
                processed_row = {
                    'file_name': Path(file_path).name,
                    'row_index': idx,
                    'source_city': city_name  # Add explicit source city tracking
                }
                
                try:
                    # Store location information
                    processed_row['city'] = row.get('city', city_name)
                    if 'location' in row:
                        processed_row['location'] = row['location']
                    
                    # Process specs data
                    if 'new_car_specs' in row and pd.notna(row['new_car_specs']):
                        specs_data = row['new_car_specs']
                        if isinstance(specs_data, str):
                            try:
                                specs_dict = json.loads(specs_data)
                            except json.JSONDecodeError:
                                try:
                                    specs_dict = ast.literal_eval(specs_data)
                                except:
                                    print(f"Error parsing new_car_specs in row {idx}")
                                    specs_dict = {}
                        else:
                            specs_dict = specs_data
                            
                        if specs_dict:
                            specs_result = extract_car_specs(specs_dict)
                            processed_row.update(specs_result)
                    
                    # Process other JSON columns
                    if 'new_car_detail' in row:
                        json_data = parse_json_safely(row['new_car_detail'])
                        processed_row.update(extract_car_details(json_data))
                    
                    if 'new_car_overview' in row:
                        json_data = parse_json_safely(row['new_car_overview'])
                        processed_row.update(extract_car_overview(json_data))
                    
                    if 'new_car_feature' in row:
                        json_data = parse_json_safely(row['new_car_feature'])
                        processed_row.update(extract_car_features(json_data))
                    
                    # Add any non-JSON columns
                    for col in df.columns:
                        if col not in ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs']:
                            processed_row[col] = row[col]
                    
                except Exception as e:
                    print(f"Error processing row {idx} in {city_name} data: {str(e)}")
                    continue
                
                all_data.append(processed_row)
                
        except Exception as e:
            print(f"Error processing file {file_path}: {str(e)}")
            continue
    
    # Create final DataFrame
    final_df = pd.DataFrame(all_data)
    
    # Verify data by city
    print("\nData verification by city:")
    print(final_df['source_city'].value_counts())
    
    # Save to CSV
    if output_csv:
        final_df.to_csv(output_csv, index=False)
        print(f"\nSaved processed data to {output_csv}")
        
        # Print column summary by city
        print("\nColumn Summary by City:")
        for city in final_df['source_city'].unique():
            city_df = final_df[final_df['source_city'] == city]
            print(f"\n{city} Data Summary:")
            print(f"Total rows: {len(city_df)}")
            
            categories = ['detail_', 'overview_', 'feature_', 'specs_']
            for prefix in categories:
                cols = [col for col in city_df.columns if col.startswith(prefix)]
                if cols:
                    print(f"\n{prefix.upper()} columns in {city}:")
                    for col in sorted(cols):
                        non_null_count = city_df[col].notna().sum()
                        print(f"- {col} ({non_null_count} non-null values)")
    
    return final_df

def main():
    """
    Main function to execute the processing for multiple cities
    """
    excel_files = [
        r"C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\jaipur_cars.xlsx"
    ]
    output_path = "processed_car_data_jaipur.csv"
    
    try:
        # Process files
        result_df = process_excel_files(excel_files, output_path)
        
        # Display statistics by city
        for city in result_df['source_city'].unique():
            city_df = result_df[result_df['source_city'] == city]
            print(f"\nStatistics for {city}:")
            print(f"Total cars: {len(city_df)}")
            
            if 'detail_price' in city_df.columns:
                print(f"\nPrice Distribution in {city}:")
                print(city_df['detail_price'].value_counts().head())
            
            if 'detail_modelYear' in city_df.columns:
                print(f"\nModel Year Distribution in {city}:")
                print(city_df['detail_modelYear'].value_counts().sort_index().head())
            
            if 'detail_transmission' in city_df.columns:
                print(f"\nTransmission Types in {city}:")
                print(city_df['detail_transmission'].value_counts())
            
            print("\n" + "="*50)
        
        # Save separate CSV files for each city
        for city in result_df['source_city'].unique():
            city_df = result_df[result_df['source_city'] == city]
            city_output = f"processed_car_data_{city.lower()}.csv"
            city_df.to_csv(city_output, index=False)
            print(f"\nSaved {city} data to {city_output}")
            
    except Exception as e:
        print(f"Error in main execution: {str(e)}")

if __name__ == "__main__":
    main()


Processing Jaipur data from C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\jaipur_cars.xlsx...
Initial data shape for Jaipur: (1120, 5)

Data verification by city:
source_city
Jaipur    1120
Name: count, dtype: int64

Saved processed data to processed_car_data_jaipur.csv

Column Summary by City:

Jaipur Data Summary:
Total rows: 1120

DETAIL_ columns in Jaipur:
- detail_bt (1120 non-null values)
- detail_centralVariantId (1120 non-null values)
- detail_ft (1120 non-null values)
- detail_km (1120 non-null values)
- detail_model (1120 non-null values)
- detail_modelYear (1120 non-null values)
- detail_oem (1120 non-null values)
- detail_owner (1120 non-null values)
- detail_ownerNo (1120 non-null values)
- detail_price (1120 non-null values)
- detail_priceFixedText (0 non-null values)
- detail_transmission (1120 non-null values)
- detail_trendingText (1120 non-null values)
- detail_variantName (1120 non-null values)

OVERVIEW_ columns in Jaipur:
- overview

In [11]:
df_jaipur=pd.read_csv("processed_car_data_jaipur.csv")
df_jaipur

,file_name,row_index,source_city,city,specs_top_mileage,specs_top_engine,specs_top_max_power,specs_top_torque,specs_top_wheel_size,specs_top_seats,...,feature_comfort_convenience,feature_interior,feature_exterior,feature_safety,feature_entertainment_communication,car_links,specs_dimensions_capacity_gross_weight,specs_engine_and_transmission_borex_stroke,specs_engine_and_transmission_compression_ratio,specs_dimensions_capacity_ground_clearance_unladen
0,jaipur_cars.xlsx,0,Jaipur,Jaipur,22.54 kmpl,1396 CC,88.73bhp,219.7Nm,16,5.0,...,"Power Steering, Power Windows Front, Power Win...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe...","Anti Lock Braking System, Centeral Locking, Po...","Cd Player, Radio, Speakers Front, Speakers Rea...",https://www.cardekho.com/used-car-details/used...,NaN,NaN,NaN,NaN
1,jaipur_cars.xlsx,1,Jaipur,Jaipur,22.38 kmpl,1197 CC,88.50bhp,113Nm,NaN,5.0,...,"Power Steering, Power Windows Front, Power Win...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Power Adjustable Exter...","Anti Lock Braking System, Brake Assist, Center...","Radio, Speakers Front, Speakers Rear, Usb Auxi...",https://www.cardekho.com/buy-used-car-details/...,1335,NaN,NaN,NaN
2,jaipur_cars.xlsx,2,Jaipur,Jaipur,23.1 kmpl,998 CC,67.04bhp,90Nm,NaN,5.0,...,"Power Steering, Power Windows Front, Power Win...","Air Conditioner, Heater, Digital Odometer, Tac...","Adjustable Head Lights, Manually Adjustable Ex...","Centeral Locking, Child Safety Locks, Day Nigh...",NaN,https://www.cardekho.com/used-car-details/used...,1250kg,73 X 82 mm,11.0:1,NaN
3,jaipur_cars.xlsx,3,Jaipur,Jaipur,18.9 kmpl,1197 CC,81.86bhp,113.75nm,NaN,5.0,...,"Power Steering, Power Windows Front, Power Win...","Air Conditioner, Heater, Digital Odometer, Tac...","Adjustable Head Lights, Power Adjustable Exter...","Anti Lock Braking System, Centeral Locking, Po...","Speakers Front, Speakers Rear, Integrated2Din ...",https://www.cardekho.com/buy-used-car-details/...,NaN,NaN,NaN,NaN
4,jaipur_cars.xlsx,4,Jaipur,Jaipur,20.51 kmpl,998 CC,67bhp,90Nm,NaN,5.0,...,"Power Steering, Power Windows Front, Power Win...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Powe...","Centeral Locking, Power Door Locks, Child Safe...","Cd Player, Radio, Speakers Front, Speakers Rea...",https://www.cardekho.com/used-car-details/used...,1350kg,69 x 72 mm,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,jaipur_cars.xlsx,1115,Jaipur,Jaipur,18.6 kmpl,1199 CC,79.4bhp,108Nm,14,5.0,...,"Power Steering, Power Windows Front, Power Win...","Air Conditioner, Heater, Adjustable Steering, ...","Fog Lights Front, Manually Adjustable Exterior...","Centeral Locking, Day Night Rear View Mirror, ...","Cd Player, Radio, Speakers Front, Speakers Rear",https://www.cardekho.com/used-car-details/used...,NaN,NaN,NaN,NaN
1116,jaipur_cars.xlsx,1116,Jaipur,Jaipur,19.7 kmpl,796 CC,46.3bhp,62Nm,12,5.0,...,Low Fuel Warning Light,"Electronic Multi Tripmeter, Glove Compartment",Manually Adjustable Exterior Rear View Mirror,"Brake Assist, Halogen Headlamps, Rear Seat Bel...",NaN,https://www.cardekho.com/used-car-details/used...,1140kg,68.5 x 72.0 mm,NaN,NaN
1117,jaipur_cars.xlsx,1117,Jaipur,Jaipur,19.17 kmpl,999 CC,71.01bhp,96Nm,16,5.0,...,"Power Steering, Power Windows Front, Power Win...","Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Power Adjustable Exter...","Anti Lock Braking System, Centeral Locking, Po...","Radio, Speakers Front, Speakers Rear, Integrat...",https://www.cardekho.com/used-car-details/used...,NaN,NaN,NaN,NaN
1118,jaipur_cars.xlsx,1118,Jaipur,Jaipur,19.7 kmpl,796 CC,46.3bhp,62Nm,12,5.0,...,Low Fuel Warning Light,"Electronic Multi Tripmeter, Glove Compartment",Manually Adjustable Exterior Rear View Mirror,"Brake Assist, Halogen Headlamps, Rear Seat Bel...","Usb Auxiliary Input, Bluetooth",https://www.ca

In [13]:
# Specify the full path with the filename, not just the directory
output_path = r"C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\processed_car_data_jaipur.csv"

# Save the DataFrame to the specified path
df_jaipur.to_csv(output_path, index=False)

print(f"CSV file has been exported to '{output_path}'")

CSV file has been exported to 'C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\processed_car_data_jaipur.csv'


In [14]:
def parse_json_safely(text: str) -> Dict:
    """
    Safely parse JSON string with multiple fallback methods
    """
    if not isinstance(text, str):
        return {}
    
    text = text.strip()
    if not text:
        return {}
        
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        try:
            return ast.literal_eval(text)
        except:
            return {}

def extract_car_details(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_detail section
    """
    result = {}
    
    # Direct fields we want to extract
    fields = ['ft', 'bt', 'km', 'transmission', 'ownerNo', 'owner', 
              'oem', 'model', 'modelYear','centralVariantId', 'variantName', 'price', 'priceFixedText', 'trendingText']
    
    for field in fields:
        if field in json_data:
            result[f'detail_{field}'] = json_data[field]
    
    return result

def extract_car_overview(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_overview section
    """
    result = {}
    
    if 'top' in json_data and isinstance(json_data['top'], list):
        for item in json_data['top']:
            if isinstance(item, dict) and 'key' in item and 'value' in item:
                key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                result[f'overview_{key}'] = item['value']
    
    return result

def extract_car_features(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_feature section
    """
    result = {}
    
    # Extract top features
    if 'top' in json_data and isinstance(json_data['top'], list):
        top_features = [item.get('value', '') for item in json_data['top'] if isinstance(item, dict)]
        result['feature_top'] = ', '.join(filter(None, top_features))
    
    # Extract detailed features by category
    if 'data' in json_data and isinstance(json_data['data'], list):
        for category in json_data['data']:
            if isinstance(category, dict) and 'heading' in category and 'list' in category:
                category_name = category['heading'].lower().replace(' & ', '_').replace(' ', '_')
                features = [item.get('value', '') for item in category['list'] if isinstance(item, dict)]
                result[f'feature_{category_name}'] = ', '.join(filter(None, features))
    
    return result

def extract_car_specs(json_data: Dict) -> Dict[str, Any]:
    """
    Extract information from new_car_specs section with proper handling of the nested structure
    """
    result = {}
    
    # Process top-level specifications if present
    if 'top' in json_data and isinstance(json_data['top'], list):
        for item in json_data['top']:
            if isinstance(item, dict) and 'key' in item and 'value' in item:
                clean_key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                result[f'specs_top_{clean_key}'] = item['value']
    
    # Process detailed specifications in the data section
    if 'data' in json_data and isinstance(json_data['data'], list):
        for category in json_data['data']:
            if isinstance(category, dict) and 'heading' in category and 'list' in category:
                # Clean the category name
                category_name = category['heading'].lower().replace(' & ', '_').replace(' ', '_')
                
                # Process the specifications list
                if isinstance(category['list'], list):
                    # Store each specification as an individual column
                    for item in category['list']:
                        if isinstance(item, dict) and 'key' in item and 'value' in item:
                            clean_key = item['key'].lower().replace(' ', '_').replace('(', '').replace(')', '')
                            result[f'specs_{category_name}_{clean_key}'] = item['value']
                    
                    # Also store the complete JSON for the category
                    result[f'specs_{category_name}_json'] = json.dumps(category['list'])
    
    return result

def process_excel_files(file_paths: List[str], output_csv: str) -> pd.DataFrame:
    """
    Process Excel files containing car data with location verification
    """
    all_data = []
    
    for file_path in file_paths:
        # Extract city name from file path
        city_name = Path(file_path).stem.split('_')[0].capitalize()
        print(f"\nProcessing {city_name} data from {file_path}...")
        
        try:
            # Read Excel file
            df = pd.read_excel(file_path)
            print(f"Initial data shape for {city_name}: {df.shape}")
            
            # Add city column if not present
            if 'city' not in df.columns:
                df['city'] = city_name
            
            # Process each row
            for idx, row in df.iterrows():
                processed_row = {
                    'file_name': Path(file_path).name,
                    'row_index': idx,
                    'source_city': city_name  # Add explicit source city tracking
                }
                
                try:
                    # Store location information
                    processed_row['city'] = row.get('city', city_name)
                    if 'location' in row:
                        processed_row['location'] = row['location']
                    
                    # Process specs data
                    if 'new_car_specs' in row and pd.notna(row['new_car_specs']):
                        specs_data = row['new_car_specs']
                        if isinstance(specs_data, str):
                            try:
                                specs_dict = json.loads(specs_data)
                            except json.JSONDecodeError:
                                try:
                                    specs_dict = ast.literal_eval(specs_data)
                                except:
                                    print(f"Error parsing new_car_specs in row {idx}")
                                    specs_dict = {}
                        else:
                            specs_dict = specs_data
                            
                        if specs_dict:
                            specs_result = extract_car_specs(specs_dict)
                            processed_row.update(specs_result)
                    
                    # Process other JSON columns
                    if 'new_car_detail' in row:
                        json_data = parse_json_safely(row['new_car_detail'])
                        processed_row.update(extract_car_details(json_data))
                    
                    if 'new_car_overview' in row:
                        json_data = parse_json_safely(row['new_car_overview'])
                        processed_row.update(extract_car_overview(json_data))
                    
                    if 'new_car_feature' in row:
                        json_data = parse_json_safely(row['new_car_feature'])
                        processed_row.update(extract_car_features(json_data))
                    
                    # Add any non-JSON columns
                    for col in df.columns:
                        if col not in ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs']:
                            processed_row[col] = row[col]
                    
                except Exception as e:
                    print(f"Error processing row {idx} in {city_name} data: {str(e)}")
                    continue
                
                all_data.append(processed_row)
                
        except Exception as e:
            print(f"Error processing file {file_path}: {str(e)}")
            continue
    
    # Create final DataFrame
    final_df = pd.DataFrame(all_data)
    
    # Verify data by city
    print("\nData verification by city:")
    print(final_df['source_city'].value_counts())
    
    # Save to CSV
    if output_csv:
        final_df.to_csv(output_csv, index=False)
        print(f"\nSaved processed data to {output_csv}")
        
        # Print column summary by city
        print("\nColumn Summary by City:")
        for city in final_df['source_city'].unique():
            city_df = final_df[final_df['source_city'] == city]
            print(f"\n{city} Data Summary:")
            print(f"Total rows: {len(city_df)}")
            
            categories = ['detail_', 'overview_', 'feature_', 'specs_']
            for prefix in categories:
                cols = [col for col in city_df.columns if col.startswith(prefix)]
                if cols:
                    print(f"\n{prefix.upper()} columns in {city}:")
                    for col in sorted(cols):
                        non_null_count = city_df[col].notna().sum()
                        print(f"- {col} ({non_null_count} non-null values)")
    
    return final_df

def main():
    """
    Main function to execute the processing for multiple cities
    """
    excel_files = [
        r"C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\kolkata_cars.xlsx"
    ]
    output_path = "processed_car_data_kolkata.csv"
    
    try:
        # Process files
        result_df = process_excel_files(excel_files, output_path)
        
        # Display statistics by city
        for city in result_df['source_city'].unique():
            city_df = result_df[result_df['source_city'] == city]
            print(f"\nStatistics for {city}:")
            print(f"Total cars: {len(city_df)}")
            
            if 'detail_price' in city_df.columns:
                print(f"\nPrice Distribution in {city}:")
                print(city_df['detail_price'].value_counts().head())
            
            if 'detail_modelYear' in city_df.columns:
                print(f"\nModel Year Distribution in {city}:")
                print(city_df['detail_modelYear'].value_counts().sort_index().head())
            
            if 'detail_transmission' in city_df.columns:
                print(f"\nTransmission Types in {city}:")
                print(city_df['detail_transmission'].value_counts())
            
            print("\n" + "="*50)
        
        # Save separate CSV files for each city
        for city in result_df['source_city'].unique():
            city_df = result_df[result_df['source_city'] == city]
            city_output = f"processed_car_data_{city.lower()}.csv"
            city_df.to_csv(city_output, index=False)
            print(f"\nSaved {city} data to {city_output}")
            
    except Exception as e:
        print(f"Error in main execution: {str(e)}")

if __name__ == "__main__":
    main()


Processing Kolkata data from C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\kolkata_cars.xlsx...
Initial data shape for Kolkata: (1381, 5)

Data verification by city:
source_city
Kolkata    1381
Name: count, dtype: int64

Saved processed data to processed_car_data_kolkata.csv

Column Summary by City:

Kolkata Data Summary:
Total rows: 1381

DETAIL_ columns in Kolkata:
- detail_bt (1381 non-null values)
- detail_centralVariantId (1381 non-null values)
- detail_ft (1381 non-null values)
- detail_km (1381 non-null values)
- detail_model (1381 non-null values)
- detail_modelYear (1381 non-null values)
- detail_oem (1381 non-null values)
- detail_owner (1381 non-null values)
- detail_ownerNo (1381 non-null values)
- detail_price (1381 non-null values)
- detail_priceFixedText (0 non-null values)
- detail_transmission (1381 non-null values)
- detail_trendingText (1381 non-null values)
- detail_variantName (1381 non-null values)

OVERVIEW_ columns in Kolkata:
- 

In [15]:
df_kolkata=pd.read_csv("processed_car_data_kolkata.csv")
df_kolkata

,file_name,row_index,source_city,city,specs_top_mileage,specs_top_engine,specs_top_max_power,specs_top_torque,specs_top_wheel_size,specs_top_seats,...,feature_interior,feature_exterior,feature_safety,feature_entertainment_communication,car_links,specs_dimensions_capacity_ground_clearance_unladen,specs_miscellaneous_cargo_volumn,specs_engine_and_transmission_compression_ratio,specs_miscellaneous_acceleration,specs_miscellaneous_top_speed
0,kolkata_cars.xlsx,0,Kolkata,Kolkata,19.16 kmpl,2494 CC,158.2bhp,213Nm,17,5.0,...,"Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Fog ...","Anti Lock Braking System, Brake Assist, Center...","Cd Player, Dvd Player, Radio, Speakers Front, ...",https://www.cardekho.com/used-car-details/used...,NaN,NaN,NaN,NaN,NaN
1,kolkata_cars.xlsx,1,Kolkata,Kolkata,22.7 kmpl,799 CC,53.64bhp,72Nm,NaN,5.0,...,"Air Conditioner, Heater, Tachometer, Electroni...","Adjustable Head Lights, Manually Adjustable Ex...","Anti Lock Braking System, Centeral Locking, Ch...","Radio, Speakers Front, Usb Auxiliary Input, Bl...",https://www.cardekho.com/buy-used-car-details/...,185mm,222,NaN,NaN,NaN
2,kolkata_cars.xlsx,2,Kolkata,Kolkata,19.03 kmpl,999 CC,71.01bhp,96Nm,NaN,5.0,...,"Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Power Adjustable Exter...","Anti Lock Braking System, Centeral Locking, Po...","Radio, Speakers Front, Speakers Rear, Integrat...",https://www.cardekho.com/used-car-details/used...,NaN,405,NaN,NaN,NaN
3,kolkata_cars.xlsx,3,Kolkata,Kolkata,17 kmpl,1197 CC,80 PS at 5200 rpm,"11.4 kgm at 4,000 rpm",14,5.0,...,"Air Conditioner, Heater, Adjustable Steering, ...","Fog Lights Rear, Power Adjustable Exterior Rea...","Centeral Locking, Power Door Locks, Child Safe...","Cd Player, Radio, Speakers Front, Speakers Rear",https://www.cardekho.com/buy-used-car-details/...,NaN,295 Lit,:1,12.96 Sec,NaN
4,kolkata_cars.xlsx,4,Kolkata,Kolkata,15.73 kmpl,1968 CC,174.33bhp,380Nm,16,5.0,...,"Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Fog ...","Anti Lock Braking System, Brake Assist, Center...","Cd Player, Radio, Audio System Remote Control,...",https://www.cardekho.com/used-car-details/used...,NaN,460-litres,NaN,8.2 Seconds,212 Kmph
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,kolkata_cars.xlsx,1376,Kolkata,Kolkata,25.24 kmpl,998 CC,65.71bhp,89Nm,NaN,5.0,...,"Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Power Adjustable Exter...","Anti Lock Braking System, Centeral Locking, Po...","Radio, Speakers Front, Speakers Rear, Integrat...",https://www.cardekho.com/used-car-details/used...,NaN,313,NaN,NaN,NaN
1377,kolkata_cars.xlsx,1377,Kolkata,Kolkata,22.74 kmpl,796 CC,47.3bhp,69Nm,NaN,5.0,...,"Air Conditioner, Heater, Digital Odometer, Ele...","Adjustable Head Lights, Manually Adjustable Ex...","Day Night Rear View Mirror, Passenger Side Rea...",NaN,https://www.cardekho.com/used-car-details/used...,NaN,177-litres,NaN,19 Seconds,140 kmph
1378,kolkata_cars.xlsx,1378,Kolkata,Kolkata,11.74 kmpl,1796 CC,186bhp,285Nm,17,5.0,...,"Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Fog ...","Anti Lock Braking System, Brake Assist, Center...","Cd Player, Cd Changer, Radio, Audio System Rem...",https://www.cardekho.com/used-car-details/used...,NaN,475-litres,9.3:1,8.8 Seconds,230km/hr
1379,kolkata_cars.xlsx,1379,Kolkata,Kolkata,18.5 kmpl,1197 CC,85.80bhp,114Nm,14,5.0,...,"Air Conditioner, Heater, Adjustable Steering, ...","Adjustable Head Lights, Fog Lights Front, Fog ...","Anti Lock Braking System, Centeral Locking, Po...","Cd Player, Radio, Speakers Front, Speakers Rea...",https://www.cardekho.com/used-car-details/used...,NaN,236-liters,NaN,15 Seconds,156 Kmph


In [16]:
# Specify the full path with the filename, not just the directory
output_path = r"C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\processed_car_data_kolkata.csv"

# Save the DataFrame to the specified path
df_kolkata.to_csv(output_path, index=False)

print(f"CSV file has been exported to '{output_path}'")

CSV file has been exported to 'C:\Users\Shiva\OneDrive\Desktop\cardekho\Dataset-20241106T053909Z-001\Dataset\processed_car_data_kolkata.csv'
